In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score
import numpy as np
import h5py
import copy
import time
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
class_names = ['Non Demented', 'Very Mild Demented', 'Mild Demented', 'Moderate Demented']
print("Class names:", class_names)

Class names: ['Non Demented', 'Very Mild Demented', 'Mild Demented', 'Moderate Demented']


In [4]:
data_dir = "dataset"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

In [5]:
# Data augmentation and normalization for training
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

In [6]:
batch_size = 32

train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms["train"])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms["test"])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model = model.to(device)

C:\Users\rakhe\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\rakhe\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Define loss function, optimizer, and early stopping criteria
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [9]:
# Early stopping settings
patience = 5
best_loss = np.inf
early_stopping_counter = 0

In [12]:
# Define training and validation function with robust error handling
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10, patience=5):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = float('inf')  # Initialize best_loss as a high value for comparison
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        try:
            for inputs, labels in tqdm(train_loader, desc="Training"):
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(train_loader.dataset)
            epoch_acc = running_corrects.double() / len(train_loader.dataset)

            print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Validation phase
            model.eval()
            val_running_loss = 0.0
            val_running_corrects = 0

            with torch.no_grad():
                for inputs, labels in tqdm(test_loader, desc="Validating"):
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    val_running_loss += loss.item() * inputs.size(0)
                    val_running_corrects += torch.sum(preds == labels.data)

            val_loss = val_running_loss / len(test_loader.dataset)
            val_acc = val_running_corrects.double() / len(test_loader.dataset)

            print(f"Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

            # Early stopping check
            if val_loss < best_loss:
                best_loss = val_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                early_stopping_counter = 0  # Reset counter if validation loss improves
            else:
                early_stopping_counter += 1
                if early_stopping_counter >= patience:
                    print("Early stopping triggered.")
                    break

            # Save best accuracy
            if val_acc > best_acc:
                best_acc = val_acc

        except Exception as e:
            print(f"Error during training or validation at epoch {epoch+1}: {e}")
            break

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [13]:
# Train the model
num_epochs = 10
model, best_acc = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs)
print(f"Best Validation Accuracy: {best_acc:.4f}")


Epoch 1/10
----------


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 161/161 [32:14<00:00, 12.02s/it]


Train Loss: 0.2492 Acc: 0.9035


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:45<00:00,  1.14s/it]


Validation Loss: 1.1234 Acc: 0.6357

Epoch 2/10
----------


...

Validation Loss: 1.5907 Acc: 0.6841

Epoch 10/10
----------


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 161/161 [43:32<00:00, 16.22s/it]


Train Loss: 0.0359 Acc: 0.9891


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:33<00:00,  1.19it/s]

Validation Loss: 1.3896 Acc: 0.7303
Best Validation Accuracy: 0.7365


In [19]:
model_path = "alzheimer_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved as {model_path}")

Model saved as alzheimer_model.pth


In [20]:
from PIL import Image

# Function to load the model and make predictions
def load_model_and_predict(image_path, model_path, class_names):
    # Load the model
    model = models.resnet50(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, len(class_names))
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()

    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image = data_transforms["test"](image).unsqueeze(0).to(device)

    # Make prediction
    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)

    return class_names[pred.item()]